### Misclassification Analysis

Compare your trained model to nSigma based method on scatterplots with TPC Signal and Beta to better understand, where model is working poorly and why.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# switch to the project directory
%cd ..
# working directory should be ../pdi

In [ ]:
import sys
import os
module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pdi.constants import PART_NAME_TO_TARGET_CODE

MODEL_BASE_DIR = "results/attention_dann_hyperparameter_tuning/sweep_118f437672375fa45c5e417106c304a1/kaon/run_25"
target_code = PART_NAME_TO_TARGET_CODE["kaon"]

save_dir = MODEL_BASE_DIR
os.makedirs(save_dir, exist_ok=True)

In [ ]:
import json
from pdi.config import Config
from pdi.engines import build_engine
from pdi.results_and_metrics import TestResults
from pdi.data.data_preparation import DataPreparation
from pdi.data.types import Split

test_results: dict[str, TestResults] = {}

with open(f"{MODEL_BASE_DIR}/config.json", 'r') as f:
    config_data = json.load(f)
config = Config.from_dict(config_data)
config.training.device = "cpu"
engine = build_engine(config, target_code, base_dir=MODEL_BASE_DIR)
data_prep = engine.get_data_prep()[0]
test_results["Model"] = engine.test(model_dirpath=MODEL_BASE_DIR)

# test_results["nSigma < 3.0"] = data_prep.get_nsigma_test_results(target_code, threshold_unscaled=3.0)

#### Extract Unwrapped and Unstandardized Test Split Data as a DataFrame
The test split data is explicitly obtained from the `CombinedDataLoader` to ensure consistency. Only the `CombinedDataLoader` has the knowledge of how to unwrap itself, and it will raise errors if the operation cannot be performed. While this could also be achieved by adding an additional method in the `DataPreparation` class, doing so would require `DataPreparation` to understand the internal structure of the `CombinedDataLoader`. This approach would also necessitate updates to `DataPreparation` whenever changes are made to the `CombinedDataLoader`. Therefore, the current approach is preferred for maintaining separation of concerns and avoiding unnecessary dependencies. It is also thousands times faster than iterating over and over dataloader and concatenating batches.

In [ ]:
test_dl = data_prep.create_dataloaders(
    {
        Split.TEST: 1 # not used
    },
    {
        Split.TEST: 1 # not used
    },
    False, False)[Split.TEST]

test_data_unwrapped = test_dl.unwrap()
print(test_data_unwrapped.shape)
test_data_unwrapped.head()

In [ ]:
import pandas as pd
import os
from pdi.data.data_exploration import generate_figure_thumbnails_from_iterator, plot_feature_combinations
from IPython.display import display
from pdi.data.constants import TARGET_COLUMN

features_to_plot = [("fP", "fTPCSignal"), ("fP", "fTRDPattern"), ("fP", "fBeta")]
saved_dataframes = {}  # Dictionary to store DataFrames

def generate_scatterplots(unwrapped_dl_df: pd.DataFrame, test_results: TestResults, features, save_dir, key):
    """Generate and save scatterplots for different conditions."""
    is_desired_particle = unwrapped_dl_df[TARGET_COLUMN] == test_results.target_code
    selected = test_results.test_metrics.binary_predictions == 1

    # Conditions are conditions for misclassified observation, not the other way around
    scatterplot_configs = [
        {
            "subdir": "all_observations",
            "data": unwrapped_dl_df,
            "condition": ((is_desired_particle & ~selected) | (~is_desired_particle & selected)),
            "legend": ("Correct prediction", "Incorrect prediction"),
            "title": f"Scatter of {{feature1}} vs {{feature2}} for all observations in {key}"
        },
        {
            "subdir": "target_observations",
            "data": unwrapped_dl_df[is_desired_particle],
            "condition": ~selected[is_desired_particle],
            "legend": ("Correct prediction", "Incorrect prediction"),
            "title": f"Scatter of {{feature1}} vs {{feature2}} for target observations in {key}"
        },
        {
            "subdir": "non_target_observations",
            "data": unwrapped_dl_df[~is_desired_particle],
            "condition": selected[~is_desired_particle],
            "legend": ("Correct prediction", "Incorrect prediction"),
            "title": f"Scatter of {{feature1}} vs {{feature2}} for non-target observations in {key}"
        }
    ]

    for config in scatterplot_configs:
        save_subdir = os.path.join(save_dir, config["subdir"])
        os.makedirs(save_subdir, exist_ok=True)
        html_element = generate_figure_thumbnails_from_iterator(
            plot_feature_combinations(
                config["data"], features, config["condition"],
                condition_legend=config["legend"],
                title_template=config["title"],
                log_scale_y=True,
            ),
            save_subdir
        )
        display(html_element)

# Generate scatterplots for each DataFrame
for key, ts in test_results.items():
    save_subdir = os.path.join(save_dir, key)
    generate_scatterplots(test_data_unwrapped, ts, features_to_plot, save_subdir, key)